In [1]:
import torch
import torch.nn as nn
import numpy as np
import torch.nn.functional as F

In [30]:
a = torch.randint(5,(1,2))
a

tensor([[1, 3]])

In [33]:
a.view(-1,a.shape[-1])

tensor([[1, 3]])

In [26]:
torch.randint(5,(1,2)).view(-1)

tensor([0, 4])

In [54]:
START=1
batch_size = 5
start_char_ids = [[START] * batch_size]
current_char_ids = torch.tensor([START for _ in range(batch_size)]).unsqueeze(0)


In [55]:
current_char_ids

tensor([[1, 1, 1, 1, 1]])

In [56]:
a = torch.randn((1,3,2))

In [57]:
a

tensor([[[-0.7952,  1.4564],
         [ 1.5980, -0.6155],
         [ 0.6012,  0.8505]]])

In [61]:
a.squeeze(0)

tensor([[-0.7952,  1.4564],
        [ 1.5980, -0.6155],
        [ 0.6012,  0.8505]])

In [62]:
prob = torch.softmax(a.squeeze(0), dim=1)

In [63]:
prob

tensor([[0.0952, 0.9048],
        [0.9015, 0.0985],
        [0.4380, 0.5620]])

In [66]:
torch.argmax(prob,dim=1).unsqueeze(0)

tensor([[1, 0, 1]])

In [71]:
torch.stack((torch.tensor([0]),torch.tensor([0])))

tensor([[0],
        [0]])

# cuda

In [3]:
torch.device('cuda')

device(type='cuda')

torch.zeros

In [30]:
torch.zeros(5,2,5)

tensor([[[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]]])

# nn.embedding

In [138]:
# an Embedding module containing 10 tensors of size 3
embedding = nn.Embedding(10, 3)

In [160]:
# a batch of 2 samples of 4 indices each
input = torch.LongTensor([[[1,2],
                          [4,3]],
                          
                          [[1,2],
                           [5,6]]])

In [161]:
input.shape

torch.Size([2, 2, 2])

In [162]:
emb = embedding(input)

In [163]:
emb.shape

torch.Size([2, 2, 2, 3])

In [164]:
emb

tensor([[[[ 0.9050, -1.0441, -1.3340],
          [-2.2785, -0.3628, -0.7464]],

         [[-1.7664,  1.4555,  1.4739],
          [ 1.9053, -0.2970, -0.6306]]],


        [[[ 0.9050, -1.0441, -1.3340],
          [-2.2785, -0.3628, -0.7464]],

         [[ 0.0059,  0.4572,  1.0697],
          [-1.0006,  0.2649,  0.6547]]]], grad_fn=<EmbeddingBackward0>)

In [167]:
emb.reshape(emb.size(0) * emb.size(1), emb.size(2), emb.size(3))

tensor([[[ 0.9050, -1.0441, -1.3340],
         [-2.2785, -0.3628, -0.7464]],

        [[-1.7664,  1.4555,  1.4739],
         [ 1.9053, -0.2970, -0.6306]],

        [[ 0.9050, -1.0441, -1.3340],
         [-2.2785, -0.3628, -0.7464]],

        [[ 0.0059,  0.4572,  1.0697],
         [-1.0006,  0.2649,  0.6547]]], grad_fn=<ReshapeAliasBackward0>)

# nn.LSTM

Params:  
squence_length: Number of hidden states  
num_layers: number of rnn layers stack on top of each other  
n_batch: the number of sequence in a batch

Outputs:  
output: the output of each hidden states  
hn: the last hidden state of each sequence in the batch  
cn: the last cell state of each sequence in the batch  


In [87]:
input_size = 3
hidden_size = 3
num_layers = 1
sequence_length = 4
n_batch = 2
rnn = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, bidirectional=True)
input = torch.randn(n_batch, sequence_length, input_size)
h0 = torch.randn(n_batch, num_layers*2, hidden_size)
c0 = torch.randn(n_batch, num_layers*2, hidden_size)
output, (hn, cn) = rnn(input, (h0, c0))

In [88]:
output.permute

<function Tensor.permute>

In [89]:
# (seq_len, batch, hidden_size)
hn

tensor([[[ 0.1273, -0.3526,  0.3220],
         [ 0.1269,  0.1439,  0.2907]],

        [[-0.3306, -0.0783,  0.0377],
         [-0.1664, -0.1468, -0.0727]]], grad_fn=<StackBackward0>)

In [90]:
# Backwards LSTM
hn[1]

tensor([[-0.3306, -0.0783,  0.0377],
        [-0.1664, -0.1468, -0.0727]], grad_fn=<SelectBackward0>)

In [91]:
# Forward LSTM
hn[0]

tensor([[ 0.1273, -0.3526,  0.3220],
        [ 0.1269,  0.1439,  0.2907]], grad_fn=<SelectBackward0>)

In [92]:
output

tensor([[[-0.1340,  0.1257,  0.2362, -0.3306, -0.0783,  0.0377],
         [ 0.0345, -0.1481,  0.1990, -0.2299, -0.3198,  0.0165],
         [ 0.1527, -0.2046,  0.2217, -0.3202, -0.3503,  0.0776],
         [ 0.1273, -0.3526,  0.3220, -0.3173, -0.4222,  0.1390]],

        [[-0.1891,  0.1778,  0.4068, -0.1664, -0.1468, -0.0727],
         [-0.0138,  0.1425,  0.2929, -0.1228,  0.0751, -0.1080],
         [ 0.1055, -0.0438,  0.4594, -0.1970, -0.2502,  0.2162],
         [ 0.1269,  0.1439,  0.2907, -0.0205, -0.1369,  0.1589]]],
       grad_fn=<TransposeBackward0>)

In [95]:
input_size = 3
hidden_size = 3
num_layers = 1
sequence_length = 4
n_batch = 2
rnn = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, bidirectional=True)
rnn(input)

(tensor([[[-0.0357,  0.1048, -0.1129, -0.1005, -0.0903,  0.3420],
          [ 0.3082, -0.1156, -0.1206,  0.0921,  0.0795,  0.2075],
          [ 0.5103,  0.0907, -0.0201, -0.0298,  0.1361,  0.1658],
          [ 0.0778, -0.1112, -0.1317,  0.0089,  0.0496,  0.0187]],
 
         [[ 0.2277,  0.1505,  0.0413, -0.1344,  0.0460,  0.2704],
          [ 0.2624,  0.3114, -0.1227, -0.1387, -0.3661,  0.1926],
          [ 0.4210,  0.2460, -0.1437,  0.0407,  0.0697,  0.2172],
          [ 0.4452, -0.0907, -0.2456,  0.1181, -0.1493,  0.1044]]],
        grad_fn=<TransposeBackward0>),
 (tensor([[[ 0.0778, -0.1112, -0.1317],
           [ 0.4452, -0.0907, -0.2456]],
  
          [[-0.1005, -0.0903,  0.3420],
           [-0.1344,  0.0460,  0.2704]]], grad_fn=<StackBackward0>),
  tensor([[[ 0.1584, -0.1729, -0.1996],
           [ 0.7648, -0.1450, -0.3888]],
  
          [[-0.3640, -0.1378,  0.7178],
           [-0.3973,  0.0749,  0.7218]]], grad_fn=<StackBackward0>)))

# nn.linear

In [43]:
input_features = 3
output_features = 1
n_batch = 4
seq_length = 3
linear = nn.Linear(in_features=input_features, out_features=output_features)
input = torch.randn(n_batch, seq_length, input_features)
output = linear(input)

In [44]:
input

tensor([[[-1.7558, -0.6634,  0.9518],
         [ 0.9306,  0.2165,  0.4547],
         [ 0.3520,  0.7379,  0.9461]],

        [[ 0.4882, -0.3100,  0.1952],
         [ 0.8007,  0.7878, -0.5927],
         [ 0.2979, -0.0604, -1.5264]],

        [[ 3.2773, -0.3718,  0.8022],
         [ 0.2837, -2.2300, -0.1480],
         [-0.8481, -1.0593, -1.3431]],

        [[ 0.1086, -1.0265,  0.8042],
         [-1.6906,  1.7651, -1.0986],
         [ 0.4211, -0.5142,  1.1259]]])

In [45]:
output

tensor([[[ 0.5643],
         [ 0.1497],
         [ 0.7433]],

        [[-0.0960],
         [ 0.1553],
         [-0.4628]],

        [[-0.6891],
         [-1.1799],
         [-0.6293]],

        [[-0.1821],
         [ 1.1910],
         [ 0.1112]]], grad_fn=<AddBackward0>)

# torch.squeeze

In [48]:
x = torch.zeros(2,1,2,1,2)

In [49]:
x

tensor([[[[[0., 0.]],

          [[0., 0.]]]],



        [[[[0., 0.]],

          [[0., 0.]]]]])

In [50]:
y = torch.squeeze(x)

In [51]:
y

tensor([[[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]]])

In [102]:
# torch.unsqueeze
x = torch.zeros(2)
print(x.shape)
x

torch.Size([2])


tensor([0., 0.])

In [103]:
y = torch.unsqueeze(x, dim=1)
print(y.shape)
y

torch.Size([2, 1])


tensor([[0.],
        [0.]])

In [104]:
y = torch.unsqueeze(x, dim=0)
print(y.shape)
y

torch.Size([1, 2])


tensor([[0., 0.]])

# torch.bmm
Performs a batch matrix-matrix product of matrices stored in input and mat2.

In [173]:
input = torch.randint(5, (2, 2, 2))

In [174]:
input

tensor([[[1, 0],
         [1, 2]],

        [[2, 3],
         [4, 1]]])

In [183]:
mat2 = torch.randint(5, (2, 2))

In [184]:
mat2

tensor([[1, 0],
        [0, 3]])

In [185]:
print(mat2.size())
mat2 = torch.unsqueeze(mat2, dim=2)
print(mat2.size())

torch.Size([2, 2])
torch.Size([2, 2, 1])


In [186]:
mat2

tensor([[[1],
         [0]],

        [[0],
         [3]]])

In [178]:
mat2.size()

torch.Size([2, 2, 1])

In [187]:
print('{} x {}'.format(input.size(), mat2.size()))
bmm = input.bmm(mat2)
bmm.size()

torch.Size([2, 2, 2]) x torch.Size([2, 2, 1])


torch.Size([2, 2, 1])

In [169]:
res = torch.bmm(input, mat2)
res.size()

torch.Size([2, 2, 1])

In [170]:
input

tensor([[[1, 4],
         [0, 3]],

        [[3, 2],
         [3, 3]]])

In [171]:
mat2

tensor([[[0],
         [4]],

        [[1],
         [3]]])

In [172]:
res

tensor([[[16],
         [12]],

        [[ 9],
         [12]]])

# torch.function.softmax

In [203]:
e_t = torch.randn(1, 2)

In [204]:
e_t

tensor([[-1.2496,  0.9797]])

In [205]:
a = torch.nn.functional.softmax(e_t, dim=1)

In [207]:
a

tensor([[0.0972, 0.9028]])

In [206]:
a.sum()

tensor(1.0000)

# torch.stack

In [54]:
a = []
for _ in range(4):
    a.append(torch.randn(5,2))
a

[tensor([[ 0.0068, -0.9010],
         [ 1.2017,  0.2322],
         [ 0.1824, -3.2524],
         [ 0.3092,  0.6805],
         [-1.2087,  0.2029]]),
 tensor([[-1.4229,  0.6015],
         [ 0.1698, -0.6921],
         [ 0.3015,  1.7531],
         [ 0.0749, -1.0463],
         [ 0.4371, -1.5216]]),
 tensor([[ 1.1616,  1.4032],
         [-0.7861,  1.6476],
         [-0.2370,  0.1055],
         [-1.6340,  0.6849],
         [-0.1305,  0.9114]]),
 tensor([[ 0.4900, -0.3961],
         [-1.6474,  2.8038],
         [-1.6253, -0.2727],
         [-0.2024,  0.3787],
         [-0.1197, -0.3842]])]

In [60]:
torch.stack(a,dim=0).shape

torch.Size([4, 5, 2])

In [ ]:
torch.stack(a,dim=1).shape

# Hadamard Product
Element Wise Product

In [17]:
input = torch.randint(5, (2, 2))
input2 = torch.randint(5, (2, 2))

In [18]:
input

tensor([[3, 3],
        [4, 0]])

In [19]:
input2

tensor([[2, 3],
        [0, 3]])

In [20]:
input * input2

tensor([[6, 9],
        [0, 0]])

# Maxpool

In [131]:
# target output size of 5
max_pool = nn.AdaptiveMaxPool1d(1)

In [42]:
input

tensor([[[ 0.4940,  0.7870, -0.6775,  0.3213,  1.0749,  0.8203, -1.8714,
           0.7055]]])

In [43]:
output

tensor([[[1.0749]]])

In [37]:
x = torch.arange(0, 5).view(1, 1, -1).float()
print(x)

tensor([[[0., 1., 2., 3., 4.]]])


# Conv1D

In [126]:
f = 2 # CS224N Number of embedding
e_char = 2
m_word = 4
m = nn.Conv1d(in_channels=e_char, out_channels=f, kernel_size=3, stride=1)
input = torch.randn(2, e_char, m_word)
output = m(input)

In [127]:
input

tensor([[[-0.9011,  0.7827,  0.3014,  1.4253],
         [ 0.4824,  0.2495,  1.8797,  0.1667]],

        [[ 2.1960, -1.8411,  0.2877,  0.9253],
         [-0.0929,  1.3117,  0.6784,  0.1913]]])

In [128]:
output.shape

torch.Size([2, 2, 2])

In [129]:
output

tensor([[[ 0.3249,  0.2966],
         [-0.5313, -0.6488]],

        [[-0.7856,  0.6592],
         [-0.2762,  0.4601]]], grad_fn=<SqueezeBackward1>)

In [102]:
output.squeeze(dim=2).shape

torch.Size([2, 2])

In [134]:
F.relu(output)

tensor([[[0.3249, 0.2966],
         [0.0000, 0.0000]],

        [[0.0000, 0.6592],
         [0.0000, 0.4601]]], grad_fn=<ReluBackward0>)

In [137]:
max_pool(output).shape

torch.Size([2, 2, 1])